In [2]:
import os
import pathlib
import json
import osc_ingest_trino as osc
import trino
import sqlalchemy
from sqlalchemy.engine import create_engine
import pandas as pd
from pyarrow import fs, csv, parquet


def _do_sql(sql, engine, verbose=False):
    if type(sql) != sqlalchemy.sql.elements.TextClause:
      #  sql = sqlalchemy.sql.text(str(sql))
         sql = sql
    if verbose:
        print(sql)
    qres = engine.execute(sql)
    res = None
    if qres.returns_rows:
        res = qres.fetchall()
        if verbose:
            print(res)
    return res

MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
s3_bucket = 'pcaf'
ingest_catalog = 'hive'
ingest_schema = 'pcaf'
sqlstring = f"trino://admin:@trino:8080/{ingest_catalog}"
engine = create_engine(sqlstring, connect_args={"http_scheme": "http"},)
engine.connect()

minio = fs.S3FileSystem( endpoint_override= 'minio:9000',
                     access_key=MINIO_ACCESS_KEY,
                     secret_key=MINIO_SECRET_KEY,
                     scheme='http')



In [6]:
sql=f"""
show tables from hive.pcaf
"""

_do_sql(sql, engine)

ProgrammingError: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SCHEMA_NOT_FOUND, message="line 2:1: Schema 'pcaf' does not exist", query_id=20231214_174346_01048_2chkx)
[SQL: 
show tables from hive.pcaf
]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [3]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.pcaf_unfccc_sovereign_emissions e
"""

#_do_sql(sql, engine)
pd.read_sql(sql, engine)

,country_iso_code,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,gdp,gdp_units,gdp_ppp,gdp_ppp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units
0,LUX,2010,1.195365e+04,kt CO2e,1.215905e+04,kt CO2e,UNFCCC,5.621399e+10,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1,NLD,2002,2.239816e+05,kt CO2e,2.182565e+05,kt CO2e,UNFCCC,4.738620e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
2,LTU,2012,1.089368e+04,kt CO2e,2.108598e+04,kt CO2e,UNFCCC,4.292745e+10,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
3,JPN,1991,1.211718e+06,kt CO2e,1.283796e+06,kt CO2e,UNFCCC,3.584421e+12,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
4,JPN,2006,1.272113e+06,kt CO2e,1.356080e+06,kt CO2e,UNFCCC,4.601663e+12,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,CAN,2018,7.133221e+05,kt CO2e,7.246158e+05,kt CO2e,UNFCCC,1.725298e+12,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1403,PRT,1996,5.795477e+04,kt CO2e,6.701282e+04,kt CO2e,UNFCCC,1.226301e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1404,PRT,2010,6.296871e+04,kt CO2e,6.952986e+04,kt CO2e,UNFCCC,2.381130e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1405,CHE,2011,5.020289e+04,kt CO2e,5.118629e+04,kt CO2e,UNFCCC,7.158881e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD


In [4]:
sql=f"""
select distinct indicator, ind from {ingest_catalog}.{ingest_schema}.oecd e
"""

#_do_sql(sql, engine)
pd.read_sql(sql, engine)

,indicator,ind
0,Domestic CO2 emissions embodied in gross exports,DTOTAL
1,Domestic CO2 emissions embodied in gross exports,D35
2,Foreign CO2 emissions embodied in gross imports,DTOTAL
3,Foreign CO2 emissions embodied in gross imports,D35


In [ ]:
sql=f"""
select distinct indicator, ind from {ingest_catalog}.{ingest_schema}.oecd e
"""

#_do_sql(sql, engine)
pd.read_sql(sql, engine)